In [26]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from tqdm import trange, tqdm
import joblib
import matplotlib.pyplot as plt
import math


In [27]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [28]:
# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=3)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_n, _) = self.rnn(x)
        out = self.fc(h_n[-1])
        return out

# Define custom dataset
class PacketCaptureDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


In [29]:
def regression_through_origin_with_correlation(x, y):
    """
    Calculate the slope of the regression line that goes through the origin
    for the given data points (x, y) and return the correlation coefficient.
    
    Parameters:
    x (array-like): Array of predictor values.
    y (array-like): Array of response values.
    
    Returns:
    tuple: The slope of the regression line through the origin and the correlation coefficient.
    """
    # Reshape x to be a 2D array with one column (required by lstsq)
    x_reshaped = np.array(x).reshape(-1, 1)
    y = np.array(y)
    
    # Use lstsq to fit a line through the origin
    m, _, _, _ = np.linalg.lstsq(x_reshaped, y, rcond=None)
    
    # m contains the slope of the line through the origin
    slope = m[0]
    
    # Calculate the predicted y values using the slope
    y_pred = slope * x
    
    # Calculate the correlation coefficient
    correlation_matrix = np.corrcoef(y, y_pred)
    correlation_coefficient = correlation_matrix[0, 1]
    
    return correlation_coefficient, slope


def normalized_slope(slope):
    return math.exp(-abs(math.log10(slope)))

def tiger_score(correlation_coef, slope):
    return normalized_slope(slope) * correlation_coef

In [30]:
def regression_through_origin_with_correlation(x, y):
    """
    Calculate the slope of the regression line that goes through the origin
    for the given data points (x, y) and return the correlation coefficient.
    
    Parameters:
    x (array-like): Array of predictor values.
    y (array-like): Array of response values.
    
    Returns:
    tuple: The slope of the regression line through the origin and the correlation coefficient.
    """
    # Reshape x to be a 2D array with one column (required by lstsq)
    x_reshaped = np.array(x).reshape(-1, 1)
    y = np.array(y)
    
    # Use lstsq to fit a line through the origin
    m, _, _, _ = np.linalg.lstsq(x_reshaped, y, rcond=None)
    
    # m contains the slope of the line through the origin
    slope = m[0]
    
    # Calculate the predicted y values using the slope
    y_pred = slope * x
    
    # Calculate the correlation coefficient
    correlation_matrix = np.corrcoef(y, y_pred)
    correlation_coefficient = correlation_matrix[0, 1]
    
    return correlation_coefficient, slope

In [31]:
def regression_through_origin_with_correlation(x, y):
    """
    Calculate the slope of the regression line that goes through the origin
    for the given data points (x, y) and return the correlation coefficient.
    
    Parameters:
    x (array-like): Array of predictor values.
    y (array-like): Array of response values.
    
    Returns:
    tuple: The slope of the regression line through the origin and the correlation coefficient.
    """
    # Reshape x to be a 2D array with one column (required by lstsq)
    x_reshaped = np.array(x).reshape(-1, 1)
    y = np.array(y)
    
    # Use lstsq to fit a line through the origin
    m, _, _, _ = np.linalg.lstsq(x_reshaped, y, rcond=None)
    
    # m contains the slope of the line through the origin
    slope = m[0]
    
    # Calculate the predicted y values using the slope
    y_pred = slope * x
    
    # Calculate the correlation coefficient
    correlation_matrix = np.corrcoef(y, y_pred)
    correlation_coefficient = correlation_matrix[0, 1]
    
    return correlation_coefficient, slope

In [32]:
# Assuming PacketCaptureDataset, RNNModel, tiger_score, and regression_through_origin_with_correlation are defined elsewhere
def load_and_normalize_data(model_name):
    data_tensors = torch.load(f'../model/data_tensors_7.5s_0.01s_large.pt')
    target_tensors = torch.load(f'../model/target_tensors_7.5s_0.01s_large.pt')
    
    if isinstance(data_tensors, list):
        data_tensors = torch.stack([torch.tensor(t) for t in data_tensors]).to(device)
    else:
        data_tensors = data_tensors.to(device)
    
    if isinstance(target_tensors, list):
        target_tensors = torch.stack([torch.tensor(t) for t in target_tensors]).to(device)
    else:
        target_tensors = target_tensors.to(device)
    
    target_mean = target_tensors.mean()
    target_std = target_tensors.std()
    target_tensors = (target_tensors - target_mean) / target_std
    
    return data_tensors, target_tensors, target_mean, target_std


def denormalize(tensor, mean, std):
    return tensor * std + mean

def prepare_datasets(data_tensors, target_tensors):
    train_data, test_data, train_targets, test_targets = train_test_split(
        data_tensors, target_tensors, test_size=0.2, random_state=42
    )
    return PacketCaptureDataset(train_data, train_targets), PacketCaptureDataset(test_data, test_targets)

def evaluate_model(model, test_dataset, mean, std):
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        test_data_tensor = torch.stack([item[0] for item in test_dataset]).to(device)
        test_targets_tensor = torch.stack([item[1] for item in test_dataset]).to(device)
        pred = model(test_data_tensor).flatten()
        pred_denorm = denormalize(pred, mean, std).cpu().numpy().flatten()
        targets_denorm = denormalize(test_targets_tensor, mean, std).cpu().numpy().flatten()
        return pred_denorm, targets_denorm


def calculate_metrics(pred_denorm, targets_denorm):
    abs_errors = np.abs(pred_denorm - targets_denorm)
    percent_abs_errors = (abs_errors / targets_denorm) * 100
    mse = np.mean((pred_denorm - targets_denorm) ** 2)

    metrics = {
        'avg_abs_error': np.mean(abs_errors),
        'avg_abs_error_normalized': np.mean(percent_abs_errors),
        'median_abs_error': np.median(abs_errors),
        'median_abs_error_normalized': np.median(percent_abs_errors),
        '98th_abs_error': np.percentile(abs_errors, 98),
        '98th_avg_abs_error': np.mean(abs_errors[abs_errors <= np.percentile(abs_errors, 98)]),
        '98th_abs_error_normalized': np.percentile(percent_abs_errors, 98),
        '98th_avg_abs_error_normalized': np.mean(percent_abs_errors[percent_abs_errors <= np.percentile(percent_abs_errors, 98)]),
        'mse': mse
    }
    return metrics, abs_errors

def calculate_regression_metrics(pred, targets, abs_errors, percentile_98th):
    slope, correlation_coef = regression_through_origin_with_correlation(pred, targets)
    pred_98th = pred[abs_errors <= percentile_98th]
    targets_98th = targets[abs_errors <= percentile_98th]
    slope_98th, correlation_coef_98th = regression_through_origin_with_correlation(pred_98th, targets_98th)
    return slope, correlation_coef, slope_98th, correlation_coef_98th

def append_results(results_df, model_name, total, delta, slope, correlation_coef, tiger_score_value, metrics, slope_98th, correlation_coef_98th, tiger_score_value_98th):
    results_df = results_df.append({
        'model_name': model_name,
        'total': total,
        'delta': delta,
        'slope': slope,
        'correlation_coef': correlation_coef,
        'tiger_score': tiger_score_value,
        'avg_abs_error': metrics['avg_abs_error'],
        'avg_abs_error_normalized': metrics['avg_abs_error_normalized'],
        'median_abs_error': metrics['median_abs_error'],
        'median_abs_error_normalized': metrics['median_abs_error_normalized'],
        '98th_abs_error': metrics['98th_abs_error'],
        '98th_avg_abs_error': metrics['98th_avg_abs_error'],
        '98th_abs_error_normalized': metrics['98th_abs_error_normalized'],
        '98th_avg_abs_error_normalized': metrics['98th_avg_abs_error_normalized'],
        'slope_98th': slope_98th,
        'correlation_coef_98th': correlation_coef_98th,
        'tiger_score_98th': tiger_score_value_98th,
        'mse': metrics['mse']
    }, ignore_index=True)
    return results_df

def plot_actual_vs_predicted(targets_denorm, pred_denorm, model_name):
    plt.figure()
    plt.scatter(targets_denorm, pred_denorm, label='Predicted vs Actual')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted for {model_name}')
    plt.legend()
    plt.show()

In [33]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=[
    'model_name', 'total', 'delta', 'slope', 'correlation_coef', 'tiger_score', 
    'avg_abs_error', 'avg_abs_error_normalized', 'median_abs_error', 
    'median_abs_error_normalized', '98th_abs_error', '98th_avg_abs_error',
    '98th_abs_error_normalized', '98th_avg_abs_error_normalized',
    'slope_98th', 'correlation_coef_98th', 'tiger_score_98th', 'mse'
])

hidden_size = 256

for filename in os.listdir('../model/large_models/large_models_256'):
    if not filename.startswith('large_epoch_') or not filename.endswith('.pt'):
        continue

    epoch = int(filename.split('_')[-1].split('.')[0])
    model_name = f'large_model_epoch_{epoch}'
    print(model_name)
    
    # Assuming 'total' and 'delta' can be derived or are fixed values for this set of models
    total = 100  # Replace with actual value or logic to derive it
    delta = 0.01  # Replace with actual value or logic to derive it
    data_tensors, target_tensors, target_mean, target_std = load_and_normalize_data(model_name)
    train_dataset, test_dataset = prepare_datasets(data_tensors, target_tensors)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    input_size = len(train_dataset[0][0][0])
    output_size = 1
    model = RNNModel(input_size, hidden_size, output_size).to(device)
    model.load_state_dict(torch.load(f'../model/large_models/large_models_256/{filename}'))
    
    pred_denorm, targets_denorm = evaluate_model(model, test_dataset, target_mean, target_std)
    
    correlation_coef, slope = regression_through_origin_with_correlation(pred_denorm, targets_denorm)
    tiger_score_value = tiger_score(correlation_coef, slope)
    
    metrics, abs_errors = calculate_metrics(pred_denorm, targets_denorm)
    slope_98th, correlation_coef_98th = calculate_regression_metrics(pred_denorm, targets_denorm, abs_errors, metrics['98th_abs_error'])
    tiger_score_value_98th = tiger_score(correlation_coef_98th, slope_98th)
    
    results_df = append_results(results_df, model_name, total, delta, slope, correlation_coef, tiger_score_value, metrics, slope_98th, correlation_coef_98th, tiger_score_value_98th)
    
    plot_actual_vs_predicted(targets_denorm, pred_denorm, model_name)
    
    print(f'Slope: {slope}, Correlation Coefficient: {correlation_coef}, Tiger Score: {tiger_score_value}, MSE: {metrics["mse"]}')
    print(f'Tiger Score 98th percentile for {model_name}: {tiger_score_value_98th}')

results_df.to_csv('large_model_evaluation_results.csv', index=False)


large_model_epoch_500


/tmp/ipykernel_36338/2527998908.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensors = torch.stack([torch.tensor(t) for t in data_tensors]).to(device)


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.15 GiB (GPU 0; 7.78 GiB total capacity; 3.85 GiB already allocated; 1.74 GiB free; 5.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF